In [4]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd


def make_fig(df):
    df['구매일'] = pd.to_datetime(df['구매일'])
    df['구매일(월)'] = df['구매일(월)'].astype('str')
    df_sort = df.sort_values('구매일')
    df_sort_drop = df_sort.drop_duplicates(subset=['ID'])
    df_sort_drop_group = df_sort_drop.groupby('구매일(월)')['주문번호'].count().reset_index()
    df_sort_drop_group.columns = ['구매날짜','신규유입']

    sales_data_monthly = df[['구매일(월)', '구매금액','결제금액']].groupby("구매일(월)").sum()


    layout = go.Layout(title='월별 매출과 신규유입',
                        yaxis=dict(title='구매금액', range=[0,550000000]),
                        yaxis2=dict(title='신규 유입',
                                    overlaying='y',
                                    side='right',
                                    ))

    fig = go.Figure(layout=layout)
    fig.add_trace(go.Bar(
            x=sales_data_monthly.index,
            y=sales_data_monthly['구매금액'],
            name="매출",
            marker_color="#56BDF4",
            offsetgroup="register",
            legendgroup="register",
            yaxis='y1'
        ))
    fig.update_layout(
        font=dict(size=20),
        legend=dict(
            x=0.8,
            y=0.9
        )
    )

    fig.add_trace(go.Scatter(x= df_sort_drop_group["구매날짜"], 
                            y= df_sort_drop_group["신규유입"],
                            name = '신규유입',
                            mode='lines+markers+text',
                            line={'color':'#FF9933',
                                'width':3,
                                'dash':'solid'
                                },
                            marker={'color':'#666699'},
                            showlegend=True,
                            yaxis='y2'))

    fig.update_xaxes(tickfont=dict(size=9.5))
    return fig


def make_page_new_customer(df):

    fig = make_fig(df)
    return [
        html.H1(children='신규 유입의 변화'),
        html.Div(children='''
            유아용품 쇼핑몰 A사의 신규 유입
        '''),
        dcc.Graph(
            id='graph_new_customer',
            figure=fig
        )
    ]
    
if __name__ == '__main__':
    import import_ipynb
    from jupyter_dash import JupyterDash
    from util.google import url_to_dataframe
    
    processed_df_link = "https://drive.google.com/file/d/1l9S5UU5f7L-OeKWhMRctCdSvVVxlFt_8/view?usp=sharing"

    ############################

    df = url_to_dataframe(processed_df_link)
    df['구매일'] = pd.to_datetime(df['구매일'])
    
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

    app = JupyterDash("6조 대쉬보드", external_stylesheets=external_stylesheets)
    
    app.layout = html.Div(make_page_new_customer(df))
    app.run_server(debug=True)
    

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
